In [1]:
from gurobipy import *
import math

In [6]:
p = {
0:[1,1],
1: [30,4],
2: [50,10]}
tar = [0,0]

In [3]:
def max(list):
    max_x = 0
    ind_x = -1
    max_y = 0
    ind_y = -1
    for i in list:
        if p[i][0] > max_x:
            max_x = p[i][0]
            ind_x = i
        if p[i][1] > max_y:
            max_y = p[i][1]
            ind_y = i
    return ind_x, ind_y
# print(max(p))

In [8]:
model = Model("Point")
x = model.addVar(vtype = GRB.INTEGER, name = 'x')
x_2 = model.addVar(vtype = GRB.INTEGER, name = 'x2')
y = model.addVar(vtype = GRB.INTEGER, name = 'y')
y_2 = model.addVar(vtype = GRB.INTEGER, name = 'y2')
dis = {}
m = {}
disToTar = {}
for i in p:
    disToTar[i] = pow(tar[0]-p[i][0],2) + pow(tar[1]-p[i][1],2)
for i in range(len(p)+1):
    dis[i] = model.addVar(vtype = GRB.CONTINUOUS, name = 'dis{}'.format(i))
    for j in range(2):
        m[i,j] = model.addVar(vtype = GRB.BINARY, name = 'm{},{}'.format(i,j))
ind_x, ind_y = max(p)
model.addConstr(p[ind_x][0] >= x, name = '(x)')
model.addConstr(p[ind_y][1] >= y, name = '(y)')
model.addConstr(tar[0] <= x, name = '(x)')
model.addConstr(tar[1] <= y, name = '(y)')

count = 0
model.addGenConstrPow( x, x_2, 2.0, name = "x2")
model.addGenConstrPow( y, y_2, 2.0, name = "y2")
for i in p:
    disToP = m[int(i),0]*(x_2+y_2)-m[int(i),0]*2*(x+y)*p[i][0]+ 2*m[int(i),0]*pow(p[i][0],2)
    model.addConstr(dis[int(i)] >= disToP + m[int(i),1]*disToTar[i], name = "(dis)")
model.addConstr(dis[len(p)] >= pow(x-tar[0],2) + pow(y-tar[1],2), name = "(dis)")

for i in p:
    model.addConstr(m[i,0] + m[i,1] == 1, name = '(type)')
    
    
lhs_t = LinExpr(quicksum(dis[i] for i in range(len(p)+1)))
model.setObjective(lhs_t,GRB.MINIMIZE)
model.params.NonConvex = 2
model.update()
# model.display()
model.optimize()
print("Optimize value = ", model.ObjVal)


Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 16 columns and 10 nonzeros
Model fingerprint: 0x0bc62549
Model has 4 quadratic constraints
Model has 2 general constraints
Variable types: 4 continuous, 12 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+02]
  QLMatrix range   [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve added 0 rows and 1197 columns
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 28 rows, 1227 columns, 3672 nonzeros
Presolved model has 2 SOS constraint(s)
Presolved model has 2 quadratic constraint(s)
Variable types: 1206 continuous, 21 integer (3 binary)

Root relaxation: objective 3.750783e+02, 45 iterations, 0.00 sec

In [9]:
for var in model.getVars(): 
    if(var.varName == 'x' or var.varName == 'y'):
        print('{name} : {val}'.format(name=var.varName, val = var.x))
    if(var.varName[0] == 'm' and var.x > 0):
        print('{name} : {val}'.format(name=var.varName, val = var.x))

x : 26.999999999999865
y : 10.0
m0,1 : 1.0
m1,0 : 1.0
m2,0 : 1.0
